In [25]:
import os
from docx import Document
import openai

f = open("first_open_ai_key","r")
open_ai_key = f.read()
f.close()

openai.api_key = open_ai_key

def split_text(text):
    max_chunk_size = 2048
    chunks = []
    current_chunk = ""
    for sentence in text.split("."):
        if len(current_chunk) + len(sentence) < max_chunk_size:
            current_chunk += sentence + "."
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + "."
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

def summarize_text(text):
    input_chunks = split_text(text)
    output_chunks = []
    for chunk in input_chunks:
        response = openai.completions.create(
            model="gpt-3.5-turbo-instruct",  
            prompt=(f"You are a professional journalist. Please summarize the following text to five points in Chinese words. Then translate these points to English and these five points should not exceed 300 English words in total:\n{chunk}\n\nSummary:"),
            max_tokens=300  
        )
        summary = response.choices[0].text.strip()
        output_chunks.append(summary)
        response = openai.completions.create(
            model="gpt-3.5-turbo-instruct", 
            prompt=(f"You are a professional journalist. Please summarize the following points to five points, these five points should not exceed 300 English words in total:\n{output_chunks}\n\nSummary:"),
            max_tokens=300  
        )
        final_summary = response.choices[0].text.strip()
    return final_summary


folder_path = 'newspaper'

for filename in os.listdir(folder_path):
    if filename.endswith('.docx'):
        file_path = os.path.join(folder_path, filename)
        doc = Document(file_path)
        full_text = []
        for para in doc.paragraphs:
            full_text.append(para.text)
        text = '\n'.join(full_text)
        
        summary = summarize_text(text)
        # print(text) 
        print(f'Finished processing {filename}')
        print(f"Summary for {filename}:\n{summary}\n")



Finished processing 推进金融高水平对外开放 为金融强国建设注入动能.docx
Summary for 推进金融高水平对外开放 为金融强国建设注入动能.docx:
1. Xi Jinping emphasizes the importance of promoting high-level financial openness to drive the construction of a strong financial nation.
2. The focus is on institutional openness, with the implementation of a negative list management system and pre-access national treatment.
3. Strengthening connectivity between domestic and foreign financial markets, facilitating cross-border investment and financing, and actively participating in international financial regulatory reforms.
4. Financial openness is a key driver of China's reform and development, and efforts have been made to increase openness in recent years.
5. To implement Xi Jinping's important speech, there must be a balance between China's modern financial system and traditional financial culture. Risk management is crucial and high-standard free trade zones should be established to support the goal of becoming a strong financial nation.
